While reading at https://jina.ai/news/scaling-test-time-compute-for-embedding-models/

I thought of creating a interactive dashboard with the latest models in that list (currently it is this image) - https://jina-ai-gmbh.ghost.io/content/images/size/w1000/2024/12/plot--1-.png

In [1]:
from IPython.display import Image

# https://jina-ai-gmbh.ghost.io/content/images/size/w1000/2024/12/plot--1-.png
Image(url='https://jina-ai-gmbh.ghost.io/content/images/size/w1000/2024/12/plot--1-.png')

In [10]:
# table = driver.find_elements(By.TAG_NAME, 'table')
# table = driver.find_element(By.CSS_SELECTOR, '.table.svelte-1txh5yn')

len(table)

0

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# # Initialize the WebDriver (ensure the WebDriver is in your PATH)
# driver = webdriver.Chrome()

# # Navigate to the MTEB leaderboard
# driver.get('https://huggingface.co/spaces/mteb/leaderboard')

# Wait for the leaderboard table to load
# wait = WebDriverWait(driver, 10)
# table = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
table = driver.find_element(By.TAG_NAME, 'table')
# Extract table rows
rows = table.find_elements(By.TAG_NAME, 'tr')

# Initialize lists to store data
model_names = []
performances = []
parameters = []
repo_urls = []

# Iterate over the rows to extract data
for row in rows[1:101]:  # Skip the header row and limit to top 100 models
    cols = row.find_elements(By.TAG_NAME, 'td')
    if len(cols) > 0:
        # Extract model name
        model_name = cols[0].text

        # Extract performance (assuming it's in the second column)
        performance = float(cols[1].text)

        # Extract parameters (assuming it's in the third column)
        parameter = int(cols[2].text.replace(',', ''))

        # Extract repository URL from the link in the first column
        repo_link = cols[0].find_element(By.TAG_NAME, 'a')
        repo_url = repo_link.get_attribute('href')

        # Append extracted data to the lists
        model_names.append(model_name)
        performances.append(performance)
        parameters.append(parameter)
        repo_urls.append(repo_url)

# Close the WebDriver
# driver.quit()

# Create a DataFrame
data = pd.DataFrame({
    'Model Name': model_names,
    'Performance': performances,
    'Parameters': parameters,
    'Repository URL': repo_urls
})

# Save to CSV for future reference
# data.to_csv('../data/mteb_top_100_models.csv', index=False)

print("Data extraction complete. Top 100 models saved to 'mteb_top_100_models.csv'.")


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"tag name","selector":"table"}
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001009db184 cxxbridge1$str$ptr + 3626716
1   chromedriver                        0x00000001009d39d4 cxxbridge1$str$ptr + 3596076
2   chromedriver                        0x0000000100440968 cxxbridge1$string$len + 89228
3   chromedriver                        0x0000000100484d50 cxxbridge1$string$len + 368756
4   chromedriver                        0x00000001004be5b4 cxxbridge1$string$len + 604376
5   chromedriver                        0x0000000100479568 cxxbridge1$string$len + 321676
6   chromedriver                        0x000000010047a1b8 cxxbridge1$string$len + 324828
7   chromedriver                        0x00000001009a69ac c

In [ ]:
import plotly.express as px

fig = px.scatter(
    data,
    x='Parameters',
    y='Performance',
    text='Model Name',
    title='Scaling Law of Embedding Models',
    labels={'Parameters': 'Number of Parameters', 'Performance': 'Average MTEB Performance on English Tasks'}
)

# Add trendline
fig = px.scatter(
    data,
    x='Parameters',
    y='Performance',
    trendline='ols',
    trendline_color_override='red',
    title='Scaling Law of Embedding Models',
    labels={'Parameters': 'Number of Parameters', 'Performance': 'Average MTEB Performance on English Tasks'}
)

# Update hover information
fig.update_traces(
    hovertemplate='<b>%{text}</b><br>Parameters: %{x}<br>Performance: %{y}'
)

# Add interactivity: clicking on a point opens the model's GitHub page
# Assuming you have a dictionary mapping model names to URLs
model_urls = {
    'Model A': 'https://github.com/user/model_a',
    'Model B': 'https://github.com/user/model_b',
    # Add other models and their URLs
}

def open_url(trace, points, state):
    if points.point_inds:
        model_name = points.trace_text[points.point_inds[0]]
        url = model_urls.get(model_name)
        if url:
            import webbrowser
            webbrowser.open(url)

fig.data[0].on_click(open_url)

# Show the plot
fig.show()
